# Validando a Clusterização do Gemini

Validando via código os resultados da Clusterização feita pelo Gemini em dados de rotatividade de funcionários publicados [nesse artigo](https://medium.com/@luizalaquini/ci%C3%AAncia-de-dados-sem-c%C3%B3digo-um-exemplo-de-clusteriza%C3%A7%C3%A3o-por-ia-generativa-dee5f7224cc2) na plataforma Medium.

**Por:** Luiza Batista Laquini

### Biblioteas e configurações

In [102]:
import pandas as pd
import numpy as np
import plotly.express as px 
import plotly.graph_objects as go
import pingouin as pg
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

import warnings
warnings.filterwarnings("ignore")

### Leitura e pré-processamento dos dados

In [103]:
# Leitura dos dados 
df = pd.read_excel('data.xlsx')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 650 entries, 0 to 649
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Data_Contratacao      650 non-null    datetime64[ns]
 1   Data_Desligamento     144 non-null    datetime64[ns]
 2   Meses_de_Servico      650 non-null    int64         
 3   Idade                 650 non-null    int64         
 4   Pontuacao_Desempenho  650 non-null    float64       
 5   Satisfacao_Trabalho   650 non-null    float64       
 6   Nivel_Cargo           650 non-null    object        
 7   Salario               650 non-null    float64       
 8   Horas_Extras          650 non-null    float64       
 9   Desligamento          650 non-null    int64         
dtypes: datetime64[ns](2), float64(4), int64(3), object(1)
memory usage: 50.9+ KB


In [104]:
df.head(3)

,Data_Contratacao,Data_Desligamento,Meses_de_Servico,Idade,Pontuacao_Desempenho,Satisfacao_Trabalho,Nivel_Cargo,Salario,Horas_Extras,Desligamento
0,2024-02-14,NaT,3,24,2.066788,2.043644,Pleno,4994.41,15.465241,0
1,2023-11-16,2024-05-14,6,43,3.256257,1.711719,Sênior,8798.34,19.036627,1
2,2023-11-16,NaT,6,45,4.827732,3.480214,Sênior,8891.02,13.326408,0


In [105]:
df_quant = df.drop(['Data_Contratacao', 'Data_Desligamento', 'Nivel_Cargo'], axis=1)
df_quant.describe().T

,count,mean,std,min,25%,50%,75%,max
Meses_de_Servico,650.0,18.690769,9.652262,3.000000,10.000000,19.000000,27.000000,35.000000
Idade,650.0,28.349231,8.014983,18.000000,22.000000,27.000000,33.000000,49.000000
Pontuacao_Desempenho,650.0,3.027463,1.174226,1.000242,1.988753,3.058911,4.022709,4.992102
Satisfacao_Trabalho,650.0,2.940577,1.148514,1.002868,1.968571,2.923458,3.964356,4.999415
Salario,650.0,6131.587062,2689.075566,2004.500000,4023.752500,5562.950000,7814.480000,11957.290000
Horas_Extras,650.0,10.098864,6.242406,0.003734,4.978000,9.413546,14.749310,41.638350
Desligamento,650.0,0.221538,0.415602,0.000000,0.000000,0.000000,0.000000,1.000000


In [106]:
df['Nivel_Cargo'].value_counts()

Nivel_Cargo
Junior    233
Pleno     225
Sênior    192
Name: count, dtype: int64

In [107]:
# Converter a coluna 'Nivel_Cargo' em colunas binárias
df_tratado = pd.get_dummies(df.drop(['Data_Contratacao', 'Data_Desligamento'], axis=1), columns=['Nivel_Cargo'], prefix='Nivel') *1
df_tratado

,Meses_de_Servico,Idade,Pontuacao_Desempenho,Satisfacao_Trabalho,Salario,Horas_Extras,Desligamento,Nivel_Junior,Nivel_Pleno,Nivel_Sênior
0,3,24,2.066788,2.043644,4994.41,15.465241,0,0,1,0
1,6,43,3.256257,1.711719,8798.34,19.036627,1,0,0,1
2,6,45,4.827732,3.480214,8891.02,13.326408,0,0,0,1
3,12,31,1.453944,1.617558,7606.22,14.209663,0,0,1,0
4,22,37,2.088917,2.763390,9236.03,11.059687,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
645,33,31,2.260088,1.539328,7940.88,6.847854,0,0,1,0
646,27,23,4.611915,2.521799,4893.95,4.583937,0,1,0,0
647,19,21,4.688376,4.641638,3838.70,11.306088,0,1,0,0
648,32,32,3.551361,3.998147,4739.81,4.343341,0,0,1,0


### Correlação Linear

In [108]:
# Correlação Linear
corr = df_tratado.corr()

# Mapa de calor das correlações lineares
fig = go.Figure()
fig.add_trace(
    go.Heatmap(
        x = corr.columns,
        y = corr.index,
        z = np.array(corr),
        text=corr.values,
        texttemplate='%{text:.2f}',
        colorscale='viridis'))
fig.update_layout(
    height = 600,
    width = 600)
fig.show()

### Normalização

Necessário normalizar os dados, pois estão em escalas diferentes

In [109]:
# Criando um objeto MinMaxScaler (min=0 e max=1)
scaler = MinMaxScaler()

# Aplicando o MinMaxScaler aos dados
df_normalizado = pd.DataFrame(scaler.fit_transform(df_tratado), columns=df_tratado.columns)

# Visualizando os dados normalizados
df_normalizado

,Meses_de_Servico,Idade,Pontuacao_Desempenho,Satisfacao_Trabalho,Salario,Horas_Extras,Desligamento,Nivel_Junior,Nivel_Pleno,Nivel_Sênior
0,0.00000,0.193548,0.267180,0.260419,0.300409,0.371362,0.0,0.0,1.0,0.0
1,0.09375,0.806452,0.565154,0.177366,0.682607,0.457141,1.0,0.0,0.0,1.0
2,0.09375,0.870968,0.958824,0.619872,0.691919,0.319990,0.0,0.0,0.0,1.0
3,0.28125,0.419355,0.113657,0.153805,0.562829,0.341205,0.0,0.0,1.0,0.0
4,0.59375,0.612903,0.272724,0.440511,0.726583,0.265547,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...
645,0.93750,0.419355,0.315604,0.134231,0.596454,0.164385,0.0,0.0,1.0,0.0
646,0.75000,0.161290,0.904760,0.380061,0.290316,0.110009,0.0,1.0,0.0,0.0
647,0.50000,0.096774,0.923914,0.910478,0.184290,0.271465,0.0,1.0,0.0,0.0
648,0.90625,0.451613,0.639080,0.749467,0.274828,0.104231,0.0,0.0,1.0,0.0


### Gráfico do Cotovelo

In [110]:
# Gráfico do cotovelo - para identificação da quantidade ideal de clusters

# Definindo o intervalo de valores de k a serem testados
k_values = range(1, 11)
inertia_values = []

# Calculando a inércia para cada valor de k
for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=0)
    kmeans.fit(df_normalizado)
    inertia_values.append(kmeans.inertia_)

# Criando um DataFrame com os resultados
df_elbow = pd.DataFrame({'k': k_values, 'inertia': inertia_values})

# Criando o gráfico do cotovelo
fig = px.line(df_elbow, x='k', y='inertia', title='Gráfico do Cotovelo', markers=True)
fig.update_layout(xaxis_title='Número de Clusters (k)', yaxis_title='Inércia')

# Exibindo o gráfico
fig.show()

### Gráfico de Silhueta

In [111]:
# Definindo o intervalo de valores de k a serem testados
k_values = range(2, 11)
silhouette_avg_values = []

# Calculando a média da pontuação de silhueta para cada valor de k
for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=0)
    cluster_labels = kmeans.fit_predict(df_normalizado)
    silhouette_avg = silhouette_score(df_normalizado, cluster_labels)
    silhouette_avg_values.append(silhouette_avg)

# Criando um DataFrame com os resultados
df_silhouette = pd.DataFrame({'k': k_values, 'silhouette_avg': silhouette_avg_values})

# Criando o gráfico de silhueta
fig = px.line(df_silhouette, x='k', y='silhouette_avg', title='Gráfico de Silhueta', markers=True)
fig.update_layout(xaxis_title='Número de Clusters (k)', yaxis_title='Média da Pontuação de Silhueta')

# Exibindo o gráfico
fig.show()

### KMEANS

In [112]:
kmeans_final = KMeans(n_clusters = 4, init = 'random', random_state=100).fit(df_normalizado)

# Identificando os clusters
kmeans_clusters = kmeans_final.labels_
df_tratado['cluster_kmeans'] = kmeans_clusters # adiciona o cluster no df TRATADO (para visualização dos dados nao-normalizados)
df_normalizado['cluster_kmeans'] = kmeans_clusters
df_tratado['cluster_kmeans'] = df_tratado['cluster_kmeans'].astype('category')
df_normalizado['cluster_kmeans'] = df_normalizado['cluster_kmeans'].astype('category')

df.head()

,Data_Contratacao,Data_Desligamento,Meses_de_Servico,Idade,Pontuacao_Desempenho,Satisfacao_Trabalho,Nivel_Cargo,Salario,Horas_Extras,Desligamento
0,2024-02-14,NaT,3,24,2.066788,2.043644,Pleno,4994.41,15.465241,0
1,2023-11-16,2024-05-14,6,43,3.256257,1.711719,Sênior,8798.34,19.036627,1
2,2023-11-16,NaT,6,45,4.827732,3.480214,Sênior,8891.02,13.326408,0
3,2023-05-20,NaT,12,31,1.453944,1.617558,Pleno,7606.22,14.209663,0
4,2022-07-24,NaT,22,37,2.088917,2.763390,Sênior,9236.03,11.059687,0


### ANOVA

Interpretação do output:

- cluster_kmeans MS: indica a variabilidade entre grupos<br>
- Within MS: indica a variabilidade dentro dos grupos<br>
- F: estatí­stica de teste (cluster_kmeans MS / Within MS)<br>
- p-unc: p-valor da estatí­stica F<br>
  - se p-valor < 0.05: pelo menos um cluster apresenta média estatisticamente diferente dos demais

In [113]:
pg.anova(dv='Meses_de_Servico', 
         between='cluster_kmeans', 
         data=df_normalizado,
         detailed=True).T

,0,1
Source,cluster_kmeans,Within
SS,0.524502,58.523198
DF,3,646
MS,0.174834,0.090593
F,1.929879,NaN
p-unc,0.123443,NaN
np2,0.008883,NaN


In [114]:
pg.anova(dv='Idade', 
         between='cluster_kmeans', 
         data=df_normalizado,
         detailed=True).T

,0,1
Source,cluster_kmeans,Within
SS,30.33196,13.051729
DF,3,646
MS,10.110653,0.020204
F,500.430406,NaN
p-unc,0.0,NaN
np2,0.699156,NaN


In [115]:
pg.anova(dv='Pontuacao_Desempenho', 
         between='cluster_kmeans', 
         data=df_normalizado,
         detailed=True).T

,0,1
Source,cluster_kmeans,Within
SS,0.540564,55.615593
DF,3,646
MS,0.180188,0.086092
F,2.092962,NaN
p-unc,0.099912,NaN
np2,0.009626,NaN


In [116]:
pg.anova(dv='Satisfacao_Trabalho', 
         between='cluster_kmeans', 
         data=df_normalizado,
         detailed=True).T

,0,1
Source,cluster_kmeans,Within
SS,0.108738,53.489061
DF,3,646
MS,0.036246,0.0828
F,0.437753,NaN
p-unc,0.726062,NaN
np2,0.002029,NaN


In [117]:
pg.anova(dv='Salario', 
         between='cluster_kmeans', 
         data=df_normalizado,
         detailed=True).T

,0,1
Source,cluster_kmeans,Within
SS,38.076338,9.29995
DF,3,646
MS,12.692113,0.014396
F,881.628936,NaN
p-unc,0.0,NaN
np2,0.8037,NaN


In [118]:
pg.anova(dv='Horas_Extras', 
         between='cluster_kmeans', 
         data=df_normalizado,
         detailed=True).T

,0,1
Source,cluster_kmeans,Within
SS,0.453965,14.135508
DF,3,646
MS,0.151322,0.021882
F,6.915473,NaN
p-unc,0.000138,NaN
np2,0.031116,NaN


In [119]:
pg.anova(dv='Desligamento', 
         between='cluster_kmeans', 
         data=df_normalizado,
         detailed=True).T

,0,1
Source,cluster_kmeans,Within
SS,42.72867,69.369792
DF,3,646
MS,14.24289,0.107384
F,132.635643,NaN
p-unc,0.0,NaN
np2,0.381171,NaN


In [120]:
pg.anova(dv='Nivel_Junior', 
         between='cluster_kmeans', 
         data=df_normalizado,
         detailed=True).T

,0,1
Source,cluster_kmeans,Within
SS,149.478462,0.0
DF,3,646
MS,49.826154,0.0
F,inf,NaN
p-unc,0.0,NaN
np2,1.0,NaN


In [121]:
pg.anova(dv='Nivel_Pleno', 
         between='cluster_kmeans', 
         data=df_normalizado,
         detailed=True).T

,0,1
Source,cluster_kmeans,Within
SS,147.115385,0.0
DF,3,646
MS,49.038462,0.0
F,inf,NaN
p-unc,0.0,NaN
np2,1.0,NaN


In [122]:
pg.anova(dv='Nivel_Sênior', 
         between='cluster_kmeans', 
         data=df_normalizado,
         detailed=True).T

,0,1
Source,cluster_kmeans,Within
SS,135.286154,0.0
DF,3,646
MS,45.095385,0.0
F,inf,NaN
p-unc,0.0,NaN
np2,1.0,NaN


### Características dos Clusters

In [123]:
# Agrupando os dados por cluster
df_agrupado = df_tratado.groupby(by=['cluster_kmeans'])

# Estatísticas Descritivas
carac_grupo = df_agrupado.describe().T
carac_grupo.to_excel("resultado.xlsx")